In [1]:
!pip install mlflow

  Using cached docker-7.1.0-py3-none-any.whl.metadata (3.8 kB)
  Using cached graphene-3.4.3-py2.py3-none-any.whl.metadata (6.9 kB)
  Using cached pyarrow-21.0.0-cp310-cp310-win_amd64.whl.metadata (3.4 kB)
  Using cached waitress-3.0.2-py3-none-any.whl.metadata (5.8 kB)
  Using cached cloudpickle-3.1.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached gitpython-3.1.45-py3-none-any.whl.metadata (13 kB)
  Using cached importlib_metadata-8.7.0-py3-none-any.whl.metadata (4.8 kB)
  Using cached protobuf-6.32.1-cp310-abi3-win_amd64.whl.metadata (593 bytes)
  Using cached pydantic-2.11.9-py3-none-any.whl.metadata (68 kB)
  Using cached python_dotenv-1.1.1-py3-none-any.whl.metadata (24 kB)
  Using cached sqlparse-0.5.3-py3-none-any.whl.metadata (3.9 kB)
  Using cached mako-1.3.10-py3-none-any.whl.metadata (2.9 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached rich-14.1.0-py3-none-any.whl.metadata (18 kB)
  Using cached blinker-1.9.0-py3-none-any.whl.metadata (1.

In [2]:
# Test mlflow

import mlflow

mlflow.set_tracking_uri("http://ec2-54-147-227-80.compute-1.amazonaws.com:5000/")

with mlflow.start_run():
    mlflow.log_param("param1", 15)
    mlflow.log_metric("metric1", 0.89)

🏃 View run burly-hog-619 at: http://ec2-54-147-227-80.compute-1.amazonaws.com:5000/#/experiments/0/runs/4c0f6b3ca0eb4d429839e79df95840b3
🧪 View experiment at: http://ec2-54-147-227-80.compute-1.amazonaws.com:5000/#/experiments/0


In [3]:
#creating baseline model

import numpy as np
import pandas as pd

In [4]:
df = pd.read_csv('https://raw.githubusercontent.com/Himanshu-1703/reddit-sentiment-analysis/refs/heads/main/data/reddit.csv')
df.head()

,clean_comment,category
0,family mormon have never tried explain them t...,1
1,buddhism has very much lot compatible with chr...,1
2,seriously don say thing first all they won get...,-1
3,what you have learned yours and only yours wha...,0
4,for your own benefit you may want read living ...,1


In [5]:
df.dropna(inplace=True)

In [6]:
df.drop_duplicates(inplace=True)

In [7]:
df = df[~(df['clean_comment'].str.strip() == '')]

In [8]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [9]:
# Define the preprocessing function
def preprocess_comment(comment):
    # Convert to lowercase
    comment = comment.lower()

    # Remove trailing and leading whitespaces
    comment = comment.strip()

    # Remove newline characters
    comment = re.sub(r'\n', ' ', comment)

    # Remove non-alphanumeric characters, except punctuation
    comment = re.sub(r'[^A-Za-z0-9\s!?.,]', '', comment)

    # Remove stopwords but retain important ones for sentiment analysis
    stop_words = set(stopwords.words('english')) - {'not', 'but', 'however', 'no', 'yet'}
    comment = ' '.join([word for word in comment.split() if word not in stop_words])

    # Lemmatize the words
    lemmatizer = WordNetLemmatizer()
    comment = ' '.join([lemmatizer.lemmatize(word) for word in comment.split()])

    return comment

In [10]:
# Apply the preprocessing function to the 'clean_comment' column
df['clean_comment'] = df['clean_comment'].apply(preprocess_comment)

In [11]:
df.head()

,clean_comment,category
0,family mormon never tried explain still stare ...,1
1,buddhism much lot compatible christianity espe...,1
2,seriously say thing first get complex explain ...,-1
3,learned want teach different focus goal not wr...,0
4,benefit may want read living buddha living chr...,1


In [12]:
import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split, cross_val_predict, StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
     

In [13]:
# Step 1: Vectorize the comments using Bag of Words (CountVectorizer)
vectorizer = CountVectorizer(max_features=10000)  # Bag of Words model with a limit of 1000 features

In [14]:
X = vectorizer.fit_transform(df['clean_comment']).toarray()
y = df['category']  # Assuming 'sentiment' is the target variable (0 or 1 for binary classification)

In [15]:
X

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [16]:
X.shape

(36793, 10000)

In [17]:
y

0        1
1        1
2       -1
3        0
4        1
        ..
37244    0
37245    1
37246    0
37247    1
37248    0
Name: category, Length: 36793, dtype: int64

In [18]:
y.shape

(36793,)

In [19]:
 # Step 2: Set up the MLflow tracking server
mlflow.set_tracking_uri("http://ec2-54-147-227-80.compute-1.amazonaws.com:5000/")

In [20]:
# Set or create an experiment
mlflow.set_experiment("RF Baseline")

2025/10/01 20:42:58 INFO mlflow.tracking.fluent: Experiment with name 'RF Baseline' does not exist. Creating a new experiment.


<Experiment: artifact_location='s3://mlflow-bucker-25/446501794395247967', creation_time=1759376578317, experiment_id='446501794395247967', last_update_time=1759376578317, lifecycle_stage='active', name='RF Baseline', tags={}>

In [21]:
!pip install boto3

  Using cached jmespath-1.0.1-py3-none-any.whl.metadata (7.6 kB)
   ---------------------------------------- 0.0/14.1 MB ? eta -:--:--
   ---------------------------------------- 14.1/14.1 MB 98.5 MB/s  0:00:00
Using cached jmespath-1.0.1-py3-none-any.whl (20 kB)

   ---------- ----------------------------- 1/4 [botocore]
   ---------- ----------------------------- 1/4 [botocore]
   ---------- ----------------------------- 1/4 [botocore]
   ---------- ----------------------------- 1/4 [botocore]
   ---------- ----------------------------- 1/4 [botocore]
   ---------- ----------------------------- 1/4 [botocore]
   ---------- ----------------------------- 1/4 [botocore]
   -------------------- ------------------- 2/4 [s3transfer]
   ---------------------------------------- 4/4 [boto3]



In [22]:
!pip install awscli

   ---------------------------------------- 0.0/4.7 MB ? eta -:--:--
   ---------------------------------------- 4.7/4.7 MB 55.7 MB/s  0:00:00
   ---------------------------------------- 0.0/570.5 kB ? eta -:--:--
   ---------------------------------------- 570.5/570.5 kB ?  0:00:00

  Attempting uninstall: rsa

    Found existing installation: rsa 4.9.1

    Uninstalling rsa-4.9.1:

      Successfully uninstalled rsa-4.9.1

  Attempting uninstall: docutils

    Found existing installation: docutils 0.22.2

    Uninstalling docutils-0.22.2:

      Successfully uninstalled docutils-0.22.2

   ------------- -------------------------- 1/3 [docutils]
   ------------- -------------------------- 1/3 [docutils]
   ------------- -------------------------- 1/3 [docutils]
   ------------- -------------------------- 1/3 [docutils]
   -------------------------- ------------- 2/3 [awscli]
   -------------------------- ------------- 2/3 [awscli]
   -------------------------- ------------- 2/3 [awscl